# init

> Router assembly for Phase 3 review routes

In [ ]:
#| default_exp routes.init

In [ ]:
#| export
from typing import List, Dict, Callable, Tuple, Optional

from fasthtml.common import APIRouter

from cjm_fasthtml_card_stack.core.models import CardStackUrls
from cjm_fasthtml_interactions.core.state_store import get_session_id

from cjm_plugin_system.core.manager import PluginManager

from cjm_transcript_review.models import ReviewUrls
from cjm_transcript_review.services.graph import GraphService
from cjm_transcript_review.routes.core import (
    WorkflowStateStore, _handle_update_title
)
from cjm_transcript_review.routes.card_stack import init_card_stack_router
from cjm_transcript_review.routes.audio import init_audio_router
from cjm_transcript_review.routes.commit import init_commit_router

## Router Assembly

In [ ]:
#| export
def init_review_routers(
    state_store:WorkflowStateStore,  # The workflow state store
    workflow_id:str,  # The workflow identifier
    prefix:str,  # Base prefix for review routes (e.g., "/workflow/review")
    audio_src_url:str="",  # Audio source route (from core routes)
    graph_service:Optional[GraphService]=None,  # Graph service for commit (None = no commit route)
    alert_container_id:str="commit-alert-container",  # ID of container for commit alerts
) -> Tuple[List[APIRouter], ReviewUrls, Dict[str, Callable]]:  # (routers, urls, routes)
    """Initialize and return all review routers with URL bundle."""
    # Create empty URL bundle (populated after routes are defined)
    urls = ReviewUrls()
    
    # Initialize card stack router
    card_stack_router, card_stack_routes = init_card_stack_router(
        state_store, workflow_id, f"{prefix}/card_stack", urls
    )
    
    # Initialize audio router
    audio_router, audio_routes = init_audio_router(
        state_store, workflow_id, f"{prefix}/audio", urls
    )
    
    # Initialize title update route
    title_router = APIRouter(prefix=prefix)
    
    @title_router.post("/update_title")
    def update_title(request, sess, document_title: str):
        """Update the document title in review state."""
        session_id = get_session_id(sess)
        _handle_update_title(state_store, workflow_id, session_id, document_title)
    
    # Populate the URL bundle using .to() on route functions
    urls.card_stack = CardStackUrls(
        nav_up=card_stack_routes["nav_up"].to(),
        nav_down=card_stack_routes["nav_down"].to(),
        nav_first=card_stack_routes["nav_first"].to(),
        nav_last=card_stack_routes["nav_last"].to(),
        nav_page_up=card_stack_routes["nav_page_up"].to(),
        nav_page_down=card_stack_routes["nav_page_down"].to(),
        update_viewport=card_stack_routes["update_viewport"].to(),
        save_width=card_stack_routes["save_width"].to(),
    )
    urls.audio_src = audio_src_url
    urls.speed_change = audio_routes["speed_change"].to()
    urls.toggle_auto_nav = audio_routes["toggle_auto_nav"].to()
    urls.replay_current = audio_routes["replay_current"].to()
    urls.update_title = update_title.to()
    
    routers = [card_stack_router, audio_router, title_router]
    all_routes = {**card_stack_routes, **audio_routes, "update_title": update_title}
    
    # Initialize commit router (optional - only if graph_service provided)
    if graph_service is not None:
        commit_router, commit_routes = init_commit_router(
            state_store, workflow_id, f"{prefix}/commit",
            graph_service, urls, alert_container_id
        )
        urls.commit = commit_routes["commit"].to()
        routers.append(commit_router)
        all_routes.update(commit_routes)
    
    return routers, urls, all_routes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()